# 2024年度 データサイエンス実習 最終課題
## 23vr008n 三宅研 修士2年 高林 秀

# レポート説明

- 選択した問題：選択肢① プロ野球Freak
- 概要・注意事項
    - 「2009年から2023年までの⽇付や対戦相⼿など5-6の指標 (説明変数)を元にホームゲームの観客数を予測する回帰モデルを作り、重要な指標を理解する」
    - 説明変数は、適宜、充⾜・削除・作成して頂いて構いません。(2020年のデータは除き、スワローズ以外のチームにして下さい)

# 問題設定
- 予測するチーム：`横浜DeNAベイスターズ`
- 目的変数：ホームゲームの観客数
- 説明変数
    - 試合日程 (年,日付)
    - 勝敗
    - スコア
    - 対戦相手
    - 先発投手
- 使用するモデル：//TODO:後で埋める

- データ元URL：https://baseball-freak.com/audience/23/baystars.html

In [50]:
from typing import Final as const
from IPython.display import display, Markdown
import pandas as pd
import numpy as np


In [51]:
# インポートするデータの条件指定
data_year_range: const[tuple] = (9, 23)
team: const[str] = "baystars"
base_url: const[str] = "https://baseball-freak.com/audience/"

In [52]:
# 指定範囲年のデータのインポート
datas: list[pd.DataFrame] = []
for year in range(data_year_range[0], data_year_range[1]+1):
    url = f"{base_url}{year:02}/{team}.html"
    print(url)
    dfs = pd.read_html(url)
    datas.append(dfs[2])
else:
    print("Done")


https://baseball-freak.com/audience/09/baystars.html
https://baseball-freak.com/audience/10/baystars.html
https://baseball-freak.com/audience/11/baystars.html
https://baseball-freak.com/audience/12/baystars.html
https://baseball-freak.com/audience/13/baystars.html
https://baseball-freak.com/audience/14/baystars.html
https://baseball-freak.com/audience/15/baystars.html
https://baseball-freak.com/audience/16/baystars.html
https://baseball-freak.com/audience/17/baystars.html
https://baseball-freak.com/audience/18/baystars.html
https://baseball-freak.com/audience/19/baystars.html
https://baseball-freak.com/audience/20/baystars.html
https://baseball-freak.com/audience/21/baystars.html
https://baseball-freak.com/audience/22/baystars.html
https://baseball-freak.com/audience/23/baystars.html
Done


In [53]:
# 年数のカラムを追加し、DataFrameを１つにまとめる。

for df, year in zip(datas,  np.arange(data_year_range[0], data_year_range[1]+1).tolist()):
    df['年'] = year

dataset = pd.concat(datas, ignore_index=True)
dataset

,日付,観客数,勝敗,スコア,対戦相手,先発投手,試合時間,球場,日付.1,年
0,4月7日(火),"20,168 人",●,1 - 5,巨人,寺原,3:45,横浜,4月7日(火),9
1,4月8日(水),"16,361 人",●,1 - 12,巨人,工藤,3:10,横浜,4月8日(水),9
2,4月9日(木),"16,691 人",●,2 - 9,巨人,ウォーランド,3:15,横浜,4月9日(木),9
3,4月10日(金),"12,791 人",○,9 - 1,ヤクルト,三浦,2:46,横浜,4月10日(金),9
4,4月11日(土),"17,817 人",●,0 - 3,ヤクルト,グリン,3:23,横浜,4月11日(土),9
...,...,...,...,...,...,...,...,...,...,...
1127,9月25日(月),"33,271 人",○,1 - 0,巨人,大貫,3:13,横浜,9月25日(月),23
1128,9月26日(火),"33,262 人",○,1 - 0,巨人,東,2:31,横浜,9月26日(火),23
1129,9月27日(水),"33,254 人",●,3 - 11,ヤクルト,坂本,3:02,横浜,9月27日(水),23
1130,9月29日(金),"33,267 人",○,5 - 3,阪神,石田,4:00,横浜,9月29日(金),23


In [54]:
# 欠損値のある行は除外する。
dataset.dropna(inplace=True)

dataset.notna().any()

日付      True
観客数     True
勝敗      True
スコア     True
対戦相手    True
先発投手    True
試合時間    True
球場      True
日付.1    True
年       True
dtype: bool

In [55]:
# 説明変数のみを残す
dataset.drop(columns=["球場", "日付.1", "試合時間",], inplace=True)
dataset


,日付,観客数,勝敗,スコア,対戦相手,先発投手,年
0,4月7日(火),"20,168 人",●,1 - 5,巨人,寺原,9
1,4月8日(水),"16,361 人",●,1 - 12,巨人,工藤,9
2,4月9日(木),"16,691 人",●,2 - 9,巨人,ウォーランド,9
3,4月10日(金),"12,791 人",○,9 - 1,ヤクルト,三浦,9
4,4月11日(土),"17,817 人",●,0 - 3,ヤクルト,グリン,9
...,...,...,...,...,...,...,...
1127,9月25日(月),"33,271 人",○,1 - 0,巨人,大貫,23
1128,9月26日(火),"33,262 人",○,1 - 0,巨人,東,23
1129,9月27日(水),"33,254 人",●,3 - 11,ヤクルト,坂本,23
1130,9月29日(金),"33,267 人",○,5 - 3,阪神,石田,23
